# Install finviz package

In [ ]:
!pip install finviz #a package that makes scrapping finviz easier
!pip install nest_asyncio #to fix the problem with 'event loop already running' introduced by newer versions of libraries installed with finviz

import nest_asyncio
nest_asyncio.apply()

     |████████████████████████████████| 1.3MB 5.5MB/s 
     |████████████████████████████████| 296kB 21.4MB/s 
     |████████████████████████████████| 143kB 15.6MB/s 
  Created wheel for finviz: filename=finviz-1.3.4-cp36-none-any.whl size=15193 sha256=23d2d001f1a047739a80cd57eb9c819fdce301f5791306898ff2f4248cf246f8
  Stored in directory: /root/.cache/pip/wheels/6d/fa/7c/4cc075d8af3f8ed083dd112b1a2b09888b9a9a2964c907291f
  Created wheel for user-agent: filename=user_agent-0.1.9-cp36-none-any.whl size=18807 sha256=dcc8e92567af98aa5a38b5b11411c63eb7623fcd4fe7caedf9b6957839e09703
  Stored in directory: /root/.cache/pip/wheels/92/80/3f/5d79277825042f2d4d447f594e3fc046f1e506f2b481d364b2
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3161 sha256=69ddd695e43d1a66ecf450791b02a7a0f38973044d280b1655e403b3b65e4943
  Stored in directory: /root/.cache/pip/wheels/d3/00/b3/32d613e19e08a739751dd6bf998cfed277728f8b2127ad4eb7
Successfully built finviz user-agent idna-ssl


# Filter good stocks

### Extract tickers
The finviz package provides filters to get tickers with specific characteristics. We can specify those characteristics or download everything.

In [ ]:
from finviz.helper_functions.save_data import export_to_db, export_to_csv
from finviz.screener import Screener
from finviz.main_func import get_news
import pandas as pd

#below are some filters that can be used to extract tickers from finviz
#----------- ta_gap_ +
#u for up; u0-20 for more specific up
#d for down; d0-20 for more specific down
 
#----------- ta_highlow50d_ +         (50 day high/low)
#nl -> new low
#a0to3h -> 0-3% above low; a0to5h -> 0-5% above low; a0to10h -> 0-10% above low
 
#----------- ta_pattern_ +
#wedgedown, wedgedown2, doublebottom, headandshouldersinv, 
 
#----------- ta_perf_ +
#4wdown -> month down; 13wdown -> quarter down
#d15u -> day -15%; 1w30u -> week -30%
 
filters = [] #when empty, takes the stocks of all the indices
print("Filtering stocks..")
Screener_obj = Screener(filters=filters, order='ticker')
print("Parsing every stock..")
Screener_obj.get_ticker_details()
 
# Export the screener results to CSV file
Screener_obj.to_csv('stocks.csv')

Filtering stocks..
Parsing every stock..


### Read tickers from .csv

In [ ]:
import pandas as pd
df=pd.read_csv("/content/stocks.csv")
df.count

### Data cleaning
Retain only desired indicators and change the representation of numbers to not cause trouble during later stages of processing.

In [ ]:
df = df[['Ticker', 'Price', 'Inst Own', 'Short Float', 'Target Price', 'Dividend', 'Beta', 'RSI (14)', 'Volatility', 'Volume', 'Market Cap', 'Earnings']]
 
missing_value_mark = -999

#replaces capitalization with its digit form
def market_cap_to_num(value):
  if value[-1] == 'M':
    return float(value[:-1])*1000000
  elif value[-1] == 'B':
    return float(value[:-1])*1000000000
  else:
    return missing_value_mark

#replaces missing volume with -1, and removes commas
def volume_to_num(value):
  if value == '-':
    return -1
  else:
    return float(value.replace(',', ''))
 
df['Ticker'] = df['Ticker'].replace('-','.') #because zacks doesn't recognize stocks with '-'. Finviz recognize both
df['Market Cap'] = df['Market Cap'].map(market_cap_to_num)
df['Volume'] = df['Volume'].map(volume_to_num)
for col_name in ['Price', 'RSI (14)', 'Beta', 'Dividend', 'Target Price']:
  df[col_name] = df[col_name].map(lambda value: float(value) if value != '-' else missing_value_mark)
for col_name in ['Short Float', 'Inst Own']:
  df[col_name] = df[col_name].map(lambda value: float(value[:-1]) if value[:-1].replace('.','',1).isdigit() else missing_value_mark)


### Filter tickers

In [ ]:
filtered_df = df

In [ ]:
#Below are some more filters we can use to constrain the ticker indicators
#'Gap' when a stock rises or falls a lot after market closes due to news
#'Float' how many stocks are available to buy. Low float -> low volatility -> lower volume
#'Short Float' > 25% problematic
#              > 40% most traders believe it will go down and sell
#'RSI' < 30 -> oversold
#      > 80 -> overbought
 
 
filtered_df = df[(df['Price'] < 100) & (df['RSI (14)'] < 45) & (df['Short Float'] < 7)]
filtered_df

,Ticker,Price,Inst Own,Short Float,Target Price,Dividend,Beta,RSI (14),Volatility,Volume,Market Cap,Earnings
4,AACQ,9.98,-999.0,-999.00,-999.00,-999.00,-999.00,-999.00,1.42% -,50087.0,7.226900e+08,-
21,ABBV,89.78,70.8,0.69,109.81,4.72,0.70,36.69,1.78% 1.99%,6307956.0,1.583600e+11,Jul 31 BMO
24,ABEO,1.89,53.2,3.15,-999.00,-999.00,1.18,28.08,6.92% 7.38%,1013187.0,1.701900e+08,-
28,ABIO,5.12,8.6,-999.00,-999.00,-999.00,2.86,42.90,7.08% 7.40%,1423616.0,3.041000e+07,-
35,AC,36.95,76.8,1.99,-999.00,0.20,1.25,43.38,2.40% 2.48%,6555.0,8.302700e+08,-
...,...,...,...,...,...,...,...,...,...,...,...,...
7506,ZIXI,5.72,69.5,6.39,-999.00,-999.00,1.23,44.17,4.57% 4.45%,521110.0,3.358800e+08,Aug 05 AMC
7514,ZOM,0.10,6.4,2.80,-999.00,-999.00,-0.28,30.51,4.56% 7.62%,26043068.0,5.823000e+07,-
7518,ZSL,7.56,-999.0,-999.00,-999.00,-999.00,-999.00,37.19,3.94% 5.90%,1327403.0,-9.990000e+02,-
7519,ZTO,31.01,40.0,2.23,-999.00,-999.00,0.31,37.15,3.33% 3.36%,3578565.0,2.404000e+10,Aug 12 AMC


### Get important news

In [ ]:
#add the 'Important News' column to our dataframe
filtered_df['Important News'] = "-"

#Creates a single string of headlines, seperated by @@@. The @@@ is just a way to mark the end of a headline.
def merge_important_headlines(headlines):
  headlines_with_seperator_signals = []
  for headline in headlines:
    headlines_with_seperator_signals.append(headline+'@@@')
  return ''.join(headlines_with_seperator_signals) 

#Keywords that make headlines important
keywords = ['offering', 'bankrupt', 'options activity', ' top ', 'implied volatility', 'covid-19', 'contract', 'merge', 'acqui', 'earnings', 'upgrade', 'downgrade', 'reiterate', 'crash', 'debt', 'lawsuit', 'alert', 'FDA', 'approval', 'patent']

tickers = filtered_df['Ticker'].tolist()

for ticker in tickers:
  important_headlines = []
  news = get_news(ticker)[:2] #keep first 5 news to keep the 'recent' ones
  for headline in list(map(lambda t: t[0], news)): #keep the headlines
    if any(x in headline.lower() for x in keywords):
      important_headlines.append(headline)
  #add important news to dataframe
  filtered_df.loc[filtered_df['Ticker'] == ticker, 'Important News'] = merge_important_headlines(important_headlines)

### Get Zacks rating

In [ ]:
import requests, time, random
from bs4 import BeautifulSoup
 
#add the 'Zacks Rating' column to our dataframe
filtered_df['Zacks Rating'] = "-"
ratings_map = {1 : 'Strong Buy', 2 : 'Buy', 3 : 'Hold', 4 : 'Sell', 5 : 'Strong Sell'}
tickers = filtered_df['Ticker'].tolist()
 
for ticker in tickers:
  print(ticker)
  user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36' #otherwise access forbidden due to non-human
  headers = {'User-Agent': user_agent}
  response = requests.get('https://www.zacks.com/stock/quote/'+ticker+'?q='+ticker, headers=headers)
  #print('response:' + str(response))
  soup = BeautifulSoup(response.content, 'html.parser')
  rank = soup.find_all('span', class_='z_rank rankrect_NA')
  if rank:   #Not rated html
    filtered_df.loc[filtered_df['Ticker'] == ticker, 'Zacks Rating'] = '-'
  else:
    rank = soup.find_all('p', class_='rank_view') #Stock html
    if rank:
      filtered_df.loc[filtered_df['Ticker'] == ticker, 'Zacks Rating'] = rank[0].text[24]
    else:
      rank = soup.find_all('span', class_='info-tooltip') #ETF html
      try:
        if rank:
          filtered_df.loc[filtered_df['Ticker'] == ticker, 'Zacks Rating'] = rank[0].next_sibling.string[1]
        else: #stock doesn't exist in zacks
          filtered_df.loc[filtered_df['Ticker'] == ticker, 'Zacks Rating'] = '-'
      except:
           filtered_df.loc[filtered_df['Ticker'] == ticker, 'Zacks Rating'] = 'ETF'
    
  
  time.sleep(random.randrange(10, 30, 10) / 1000) #wait a bit to appear more human (ms)

### Get Yahoo Finance rating

In [ ]:
#add the 'Yahoo Finance Rating' column to our dataframe
filtered_df['Yahoo Finance Rating'] = "-"
tickers = filtered_df['Ticker'].tolist()

for ticker in tickers:
  print(ticker)
  user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36' #otherwise access forbidden due to non-human
  headers = {'User-Agent': user_agent}
  response = requests.get('https://finance.yahoo.com/quote/'+ticker+'?p='+ticker+'&.tsrc=fin-srch', headers=headers)
  #print('response:' + str(response))
  soup = BeautifulSoup(response.content, 'html.parser')
  rank = soup.find('div', class_='Pos(a) T(-2px) W(3px) H(12px) Bdrs(4px) Bgc($primaryColor) ')
  if rank == None: #some are locked
    filtered_df.loc[filtered_df['Ticker'] == ticker, 'Yahoo Finance Rating'] = "-"
  elif rank['style'][-3:] == 'px;':
    filtered_df.loc[filtered_df['Ticker'] == ticker, 'Yahoo Finance Rating'] = "Undervalued"
  elif rank['style'][-3:] == '0%;':
    filtered_df.loc[filtered_df['Ticker'] == ticker, 'Yahoo Finance Rating'] = "Near Fair Value"
  elif rank['style'][-3:] == 'x);':
    filtered_df.loc[filtered_df['Ticker'] == ticker, 'Yahoo Finance Rating'] = "Overvalued"
    
  time.sleep(random.randrange(2, 6, 1) / 100) #wait a bit to appear more human

### Restore original missing value symbol

In [ ]:
filtered_df = filtered_df.replace({-999: '-'})

### Save to .csv

In [ ]:
from google.colab import files
from datetime import datetime
 
date = datetime.now().strftime('%d-%m-%Y')
filename = "stocks_"+date+".csv"
 
filtered_df.to_csv("/content/"+filename)
 
#download file
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Find upgrades in Zacks
This section assumes that there is a .csv file with the Zacks rating from the past.

### Extract stocks from Finviz

In [ ]:
from finviz.helper_functions.save_data import export_to_db, export_to_csv
from finviz.screener import Screener
from finviz.main_func import get_news
import pandas as pd

filters = [] 
print("Filtering stocks..")
Screener_obj = Screener(filters=filters, order='ticker')
print("Parsing every stock..")
Screener_obj.get_ticker_details()

Screener_obj.to_csv('stocks.csv')

df=pd.read_csv("/content/stocks.csv")
df=df[['Ticker']]

### Extract current Zacks rating

In [ ]:
import requests, time, random
from bs4 import BeautifulSoup

#add the 'Zacks Rating' column to our dataframe
df['Zacks Rating'] = "-"
tickers = df['Ticker'].tolist()

#for zacks we need to change the user agent otherwise it detects that it's a spot scrapping information
for ticker in tickers:
  user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36' #otherwise access forbidden due to non-human
  headers = {'User-Agent': user_agent}
  response = requests.get('https://www.zacks.com/stock/quote/'+ticker+'?q='+ticker, headers=headers)
  #print('response:' + str(response))
  soup = BeautifulSoup(response.content, 'html.parser')

  print(ticker)
  rank = soup.find_all('span', class_='z_rank rankrect_NA')
  if rank:   #Not rated html
    df.loc[df['Ticker'] == ticker, 'Zacks Rating'] = '-'
  else:
    rank = soup.find_all('p', class_='rank_view') #Stock html
    if rank:
      df.loc[df['Ticker'] == ticker, 'Zacks Rating'] = rank[0].text[24]
    else:
      rank = soup.find_all('span', class_='info-tooltip') #ETF html
      if rank:
        df.loc[df['Ticker'] == ticker, 'Zacks Rating'] = rank[0].next_sibling.string[1]
      else: #stock doesn't exist in zacks
        df.loc[df['Ticker'] == ticker, 'Zacks Rating'] = '-' 
    
  
  time.sleep(random.randrange(10, 30, 10) / 1000) #wait a bit to appear more human (ms)

#use to create the file for the first time
#df['Fresh'] = False
#df[['Fresh', 'Ticker', 'Zacks Rating']].to_csv("/content/zacks_ratings.csv")

### Compare with previous Zacks ratings

In [ ]:
from google.colab import files

files.upload() #choose the file with the old rating on local computer

prev_df = pd.read_csv("/content/zacks_ratings.csv")

missing_value_mark = -999

df['Zacks Rating'] = df['Zacks Rating'].map(lambda value: float(value) if value != '-' else missing_value_mark)     
prev_df['Zacks Rating'] = prev_df['Zacks Rating'].map(lambda value: float(value) if value != '-' else missing_value_mark)           
df['Fresh'] = prev_df['Zacks Rating'] - df['Zacks Rating'] 
df['Fresh'] = df['Fresh'].apply(lambda value: True if value > 0 else False)

#convert back to '-' to read easier
df['Zacks Rating'] = df['Zacks Rating'].map(lambda value: value if value != missing_value_mark else '-')
df.to_csv("/content/updated_zacks_ratings.csv")

# Parallel zack fetch test

In [ ]:
from multiprocessing import Process, Pool
from functools import partial
import time, requests

def process_zack_response(response):
  
  rating = ''

  soup = BeautifulSoup(response.content, 'html.parser')
  rank = soup.find_all('span', class_='z_rank rankrect_NA')
  if rank:   #Not rated html
    rating = '-'
  else:
    rank = soup.find_all('p', class_='rank_view') #Stock html
    if rank:
      rating = rank[0].text[24]
    else:
      rank = soup.find_all('span', class_='info-tooltip') #ETF html
      if rank:
        rating = rank[0].next_sibling.string[1]
      else: #stock doesn't exist in zacks
        rating = '-' 
  
  return rating

def request_wrapper(ticker):
  user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36' #otherwise access forbidden due to non-human
  headers = {'User-Agent': user_agent}
  url = 'https://www.zacks.com/stock/quote/'+ticker+'?q='+ticker
  response = requests.get(url, headers = headers)
  rating = process_zack_response(response)
  return [ticker, rating]

df['Zacks Rating'] = "-"
tickers = df['Ticker'][:1000].tolist()
pool = Pool(processes = multiprocessing.cpu_count())
results = pool.map(request_wrapper, tickers)

for result in results:
  ticker = result[0]
  rating = result[1]
  df.loc[df['Ticker'] == ticker, 'Zacks Rating'] = rating

# Stock alerts
In this section we can add alerts based on price or other indicators provided by Yahoo Finance. From Yahoo Finance because it updates free information regularly.

### Monitor ticker price (Yahoo finance)

In [ ]:
import requests, time, operator, re
from bs4 import BeautifulSoup

def print_on_same_line(price_dict):
  output=''
  for k, v in price_dict.items():
    output += k + ':' + str(v) + '  '
  print('\r', end='')
  print(output, end='')

def get_ticker_prices(tickers):

  price_dict = dict.fromkeys(tickers)
  
  for ticker in tickers:
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36' #otherwise access forbidden due to non-human
    headers = {'User-Agent': user_agent}
    response = requests.get('https://finance.yahoo.com/quote/'+ticker+'?p='+ticker+'&.tsrc=fin-srch', headers=headers)
    #print('response:' + str(response))
    soup = BeautifulSoup(response.content, 'html.parser')
    rank = soup.find('span', class_='Trsdu(0.3s) Fw(b) Fz(36px) Mb(-4px) D(ib)')
    price_dict.update({ticker : rank.text})

  return price_dict

def process_alerts(alerts, price_dict):

  alert_tickers=[] #tickers that need to have an alert executed

  split_alerts = [] #[['BNGO', '<', '0.5'], ...]
  for alert in alerts:
    if '<' in alert:
      if '=' in alert:
        split_alerts.append(re.split('(<=)', alert))
      else:
        split_alerts.append(re.split('(<)', alert))
    elif '>' in alert:
      if '=' in alert:
        split_alerts.append(re.split('(>=)', alert))
      else:
        split_alerts.append(re.split('(>)', alert))
    else:
      split_alerts.append(re.split('(=)', alert))
  
  for ticker, condition, price in split_alerts:
    if condition == '<=':
      if operator.le(price_dict[ticker], float(price)):
        alert_tickers.append(''.join([ticker, condition, price]))
    if condition == '<':
      if operator.lt(price_dict[ticker], float(price)):
        alert_tickers.append(''.join([ticker, condition, price]))
    if condition == '>=':
      if operator.ge(price_dict[ticker], float(price)):
        alert_tickers.append(''.join([ticker, condition, price]))
    if condition == '>':
      if operator.gt(price_dict[ticker], float(price)):
        alert_tickers.append(''.join([ticker, condition, price]))
    if condition == '=':
      if operator.eq(price_dict[ticker], float(price)):
        alert_tickers.append(''.join([ticker, condition, price]))

  return alert_tickers

def notify_for_alert(alert_tickers):


def monitor_tickers(tickers, alerts):

  tickers = [ticker.upper() for ticker in tickers] #make tickers uppercase
  alerts = alerts 

  while True:
    
    price_dict = get_ticker_prices(tickers)
    alert_tickers = process_alerts(alerts, price_dict)
    for alert_ticker in alert_tickers:
      notify_for_ticker_alert(alert_ticker)
    print_on_same_line(price_dict)
    time.sleep(5) 
    
 


In [ ]:
#example of monitoring some tickers
monitor_tickers(['BNGO', 'ATNM'], ['AVGR>0.35','BNGO=0.427','TSLA<=700'])

### Send alerts to email (under experimentation)

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

with open('/content/drive/My Drive/mailjet.txt') as f:
    mailjet_contents = f.readlines()

mailjet_contents = [x.strip() for x in mailjet_contents]


from mailjet_rest import Client
import os

api_key = mailjet_contents[0]
api_secret = mailjet_contents[1]
mailjet = Client(auth=(api_key, api_secret), version='v3.1')
data = {
  'Messages': [
    {
      "From": {
        "Email": "christos.pylianidis@wur.nl",
        "Name": "Christos"
      },
      "To": [
        {
          "Email": "pylianidis@gmail.com",
          "Name": "Christos"
        }
      ],
      "Subject": message,
      "TextPart": "",
      "CustomID": "AppGettingStartedTest"
    }
  ]
}
result = mailjet.send.create(data=data)
print(result.status_code)
print(result.json())
